In [1]:
# importing libraries

import pandas as pd
import re
import os
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import ast

pd.set_option('display.max_colwidth', None)

### Question 1

In [ ]:
# function to print file contents
def printFileContent(path1, path2):
    for file in os.listdir(path1)[:5]:
        f1 = open(f'{path1}/{file}', 'r')
        f2 = open(f"{path2}/{file}", 'r')
        print(f"\nContent of file {file} before text extraction\n")
        print(f1.read())    
        print(f"\nContent of file {file} after text extraction\n")
        print(f2.read())
        print("\n-----------------------------------------------------------\n")
        f1.close()
        f2.close()

In [ ]:
# function to extract text between tags from all files
def extractText(path):
    for file in os.listdir(path):
        f = open(f"{path}/{file}", 'r')
        text = f.read() 
        f.close()
        x = re.search("((?<=\<TITLE\>)(.|\n)*(?=\<\/TITLE\>))", text)
        y = re.search("((?<=\<TEXT\>)(.|\n)*(?=\<\/TEXT\>))", text)

        z = x.group(0).strip() + ' ' + y.group().strip()
        z = " ".join(z.split())
        f = open(f"{path}/{file}", 'w')
        f.write(z)
        f.close()

In [ ]:
path1 = "./Assignment1/CSE508_Winter2023_Dataset1/CSE508_Winter2023_Dataset"
path2 = "./Assignment1/CSE508_Winter2023_Dataset/CSE508_Winter2023_Dataset"
# path1 = "Desktop/IR/Assignments/CSE508_Winter2023_Dataset_1"
# path2 = "Desktop/IR/Assignments/CSE508_Winter2023_Dataset_2"

In [ ]:
# Q1) i
# extractText(path2)
printFileContent(path1, path2) 

In [ ]:
# function to lowercase the text
def toLowerCase(path,df):
    for file in os.listdir(path):
        f = open(f"{path2}/{file}", 'r')
        text = f.read()
        lower_text = text.lower()
        df.loc[len(df.index)] = [file,text,lower_text]
        f.close()

In [ ]:
# Q2) ii) 1
lowercase_df = pd.DataFrame(columns = ['Doc Id','Before Lowercase','After Lowercase'])
toLowerCase(path2,lowercase_df)
lowercase_df.set_index('Doc Id',inplace = True)
lowercase_df.head()

In [ ]:
# function to tokenize the text
def tokenizer(path,df,prev_df):
    for file in os.listdir(path):
        f = open(f"{path2}/{file}", 'r')
        text = f.read()
        tokens = word_tokenize(text)
        tokenized_text = str(tokens)
        df.loc[len(df.index)] = [file,prev_df.at[file,'After Lowercase'],tokenized_text]
        f.close()

In [ ]:
# Q2) ii) 2
tokenize_df = pd.DataFrame(columns = ['Doc Id','Before Tokenization','After Tokenization'])
tokenizer(path2,tokenize_df,lowercase_df)
tokenize_df.set_index('Doc Id',inplace = True)
tokenize_df.head()

In [ ]:
# function to remove stop words
def removeStopWords(path,df,prev_df):
    for file in os.listdir(path):
        tokens = ast.literal_eval(prev_df.at[file,'After Tokenization'])
        remove_punc = [t for t in tokens if not t in stopwords.words("english")]
        res = str(remove_punc)
        df.loc[len(df.index)] = [file,prev_df.at[file,'After Tokenization'],res]

In [ ]:
# Q2) ii) 3
stopwords_df = pd.DataFrame(columns = ['Doc Id','Before Stopwords Removal','After Stopwords Removal'])
removeStopWords(path2,stopwords_df,tokenize_df)
stopwords_df.set_index('Doc Id',inplace = True)
stopwords_df.head()

In [2]:
# function to remove punctuation
punctuations = string.punctuation
punc = [ p for p in punctuations]

def removePunctuation(path,df,prev_df):
    for file in os.listdir(path):
        tokens = ast.literal_eval(prev_df.at[file,'After Stopwords Removal'])
        res = [t for t in tokens if t not in punc]
        res = str(res)
        df.loc[len(df.index)] = [file,prev_df.at[file,'After Stopwords Removal'],res]

In [ ]:
# Q2) ii) 4
pucntuation_df = pd.DataFrame(columns = ['Doc Id','Before Punctuation Removal','After Punctuation Removal'])
removePunctuation(path2,pucntuation_df,stopwords_df)
pucntuation_df.set_index('Doc Id',inplace = True)
pucntuation_df.head()

In [ ]:
# function to remove punctuation
def removeBlanks(path,df,prev_df):
    for file in os.listdir(path):
        tokens = ast.literal_eval(prev_df.at[file,'After Punctuation Removal'])
        res = [ t.strip() for t in tokens if len(t.strip()) != 0]
        res = str(res)
        df.loc[len(df.index)] = [file,prev_df.at[file,'After Punctuation Removal'],res]

In [ ]:
# Q2) ii) 5
blanks_df = pd.DataFrame(columns = ['Doc Id','Before Blank Space Tokens Removal',
                                    'After Blank Space Tokens Removal'])
removeBlanks(path2,blanks_df,pucntuation_df)
blanks_df.set_index('Doc Id',inplace = True)
blanks_df.head()

In [ ]:
# saving final pre-processed data
final_df = blanks_df.drop('Before Blank Space Tokens Removal', axis=1)
final_df.rename(columns = {'After Blank Space Tokens Removal':'Contents of file after Preprocessing'}, 
                inplace = True)
final_df.head()

In [ ]:
final_df.to_csv("Desktop/final_df.csv")